# Podemos usar ML con inferencia por fast api, util para dispositivos que no pueden ejecutar estos modelos o deserializar estos modelos en estos dispositivos

In [2]:
pip -q install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


In [13]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return({"message": "Hello, World!"})

In [14]:
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    price: float
    is_offer: bool = False

@app.post("/items/")
def create_item(item: Item):
    return {"item_name": item.name, "item_price": item.price}

API para inferencia de ML 

In [15]:
from pydantic import BaseModel, Field

# Validacion de parametros de entrada
class IrisFeatures(BaseModel):
    sepal_lenght: float = Field( ... , ge=0, description="Longitud del sepalo")
    sepal_width: float = Field( ... , ge=0, description="Ancho del sepalo")
    petal_lenght: float = Field( ... , ge=0, description="Longitud del petalo")
    petal_width: float = Field( ... , ge=0, description="Ancho del petalo")



In [16]:
# Respuesta del modelo
class PredictResponse(BaseModel):
    prediction: int
    species: str
    confidence: str


In [17]:
from fastapi import FastAPI
import joblib
import numpy as np

# Deserializamos el modelo, el Api y el mapeo de las predicciones a especies

app = FastAPI(title="Iris Classification API")

model = joblib.load("iris_model.joblib")

species_map = {
            0 : "Setosa",
            1 : "Versicolor",
            2 : "equisde"
            }

In [18]:
@app.post("/predict", response_model=PredictResponse)

def predict(features: IrisFeatures):
    # Preparar datos para el modelo
    X = np.array ([[features.sepal_lenght, features.sepal_width,
                    features.petal_lenght, features.petal_width]])
    
    # Realizar prediccion
    prediction = model.predict(X)[0]
    confidence = model.predict_proba(X)[0].max()

    return {
        "prediction": int(prediction),
        "species" : species_map[prediction],
        "confidence" : float(confidence)
    }


Consideraciones en produccion:
- Validacion estricta de entrada
- Autenticacion y autorizacion para controlar acceso (para costos computacionales altos checar quien puede entrar)
- Rate limiting para prevenir abuso (tiene que ser a nivel de usuario)
- Manejo de errores y logging para monitoreo
- Batch predictions para multiples muestras
- Monitoreo de predicciones para detectar drift del modelo (El modelo cambia porque el modelo cambia o las predicciones cambian, tenemos que actualizar nuestros modelos)
- Containeraizacion (Docker) para despliegue consistente (Usar Docker para estandarizar las dependencias de sus entornos)
- Recoleccion de nuevos datos para re-entrenamiento continuo
